# Machine learning application: Forecasting wind power

<table>
  <tr><td>
    <img src="https://github.com/dmatrix/mlflow-workshop-part-3/raw/master/images/wind_farm.jpg"
         alt="Keras NN Model as Logistic regression"  width="800">
  </td></tr>
</table>

In this notebook, you will use the MLflow Model Registry to build a machine learning application that forecasts the daily power output of a [wind farm](https://en.wikipedia.org/wiki/Wind_farm). Wind farm power output depends on weather conditions: generally, more energy is produced at higher wind speeds. Accordingly, the machine learning models used in the notebook predict power output based on weather forecasts with three features: `wind direction`, `wind speed`, and `air temperature`.

*This notebook uses altered data from the [National WIND Toolkit dataset](https://www.nrel.gov/grid/wind-toolkit.html) provided by NREL, which is publicly available and cited as follows:*

*Draxl, C., B.M. Hodge, A. Clifton, and J. McCaa. 2015. Overview and Meteorological Validation of the Wind Integration National Dataset Toolkit (Technical Report, NREL/TP-5000-61740). Golden, CO: National Renewable Energy Laboratory.*

*Draxl, C., B.M. Hodge, A. Clifton, and J. McCaa. 2015. "The Wind Integration National Dataset (WIND) Toolkit." Applied Energy 151: 355366.*

*Lieberman-Cribbin, W., C. Draxl, and A. Clifton. 2014. Guide to Using the WIND Toolkit Validation Code (Technical Report, NREL/TP-5000-62595). Golden, CO: National Renewable Energy Laboratory.*

*King, J., A. Clifton, and B.M. Hodge. 2014. Validation of Power Output for the WIND Toolkit (Technical Report, NREL/TP-5D00-61714). Golden, CO: National Renewable Energy Laboratory.*

*(This notebook is from the [Spark + AI Summit keynote demo](https://databricks.com/session_eu19/1-simplifying-model-management-with-mlflow), originally written by Corey Zumar and later modifed, modularized and shortened for MLflow tutorial by Jules S. Damji)*

## Load the dataset

The following cells load a dataset containing weather data and power output information for a wind farm in the United States. The dataset contains `wind direction`, `wind speed`, and `air temperature`, features sampled every eight hours (once at `00:00`, once at `08:00`, and once at `16:00`), as well as daily aggregate power output (`power`), over several years.

Run the notebook to bring defined classes for models and utilities into this notebook scope

In [0]:
%run ./classes_init

In [0]:
csv_path = "https://raw.githubusercontent.com/dmatrix/mlflow-workshop-part-3/master/src/data/windfarm_data.csv"
wind_farm_data = Utils.load_data(csv_path, index_col=0)

Display a sample of the data for reference.

In [0]:
wind_farm_data.head(5)

,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
2014-01-01,4.702022,106.742590,4.743292,7.189482,100.41638,6.593833,8.172301,99.28800,5.967206,1959.3535
2014-01-02,7.695733,98.036705,6.142716,9.977118,94.03181,4.383676,9.690135,204.25444,1.696528,1266.6239
2014-01-03,9.608235,274.061200,10.514304,10.840864,242.87563,16.869741,8.991079,250.26830,12.038399,7545.6797
2014-01-04,6.955563,257.910220,7.189170,5.317223,254.26170,9.069233,3.021174,284.06537,4.590843,3791.0408
2014-01-05,0.830547,265.394400,4.263086,2.480239,104.79496,3.042063,4.227131,263.41690,3.899182,880.6115


In [0]:
wind_farm_data.describe()

,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,9.613588,198.381353,5.001413,12.827786,191.538394,5.136628,13.991459,202.051406,6.072578,2424.978880
std,5.187017,81.767411,2.940271,6.536972,83.572022,3.016934,6.862011,88.747683,2.585946,2035.838491
min,-6.172769,14.527682,0.232208,-3.989666,14.087781,0.519220,-4.087519,5.659133,0.561177,207.065570
25%,6.228526,111.309685,2.863503,7.932104,108.053078,2.897562,8.576865,111.450595,4.299781,947.100430
50%,8.688278,214.461780,4.265017,11.178736,174.028730,4.356376,12.392289,245.210940,5.785738,1674.430900
75%,13.045183,276.894330,6.263791,17.898135,281.361380,6.745475,19.687266,275.937150,7.550025,3274.856450
max,27.039269,352.764130,19.162184,30.118828,351.428300,23.394577,32.521378,349.589100,18.859266,10774.853000


### Get Training and Validation Data

In [0]:
X_train, y_train = Utils.get_training_data(wind_farm_data)
val_x, val_y = Utils.get_validation_data(wind_farm_data)

# Train a power forecasting model and track it with MLflow

The following cells train a neural network in Keras to predict power output based on the weather features in the dataset. MLflow is used to track the model's hyperparameters, performance metrics, source code, and artifacts.

Train the model and use MLflow to track its parameters, metrics, artifacts, and source code.

Define a power forecasting model in Keras, and create three different models with different configuratons and tuning parameters

<table>
  <tr><td>
    <img src="https://github.com/dmatrix/mlflow-workshop-part-3/raw/master/images/nn_linear_regression.png"
         alt="Keras NN Model as Logistic regression"  width="800">
  </td></tr>
</table>

Iterate over three different set of tunning parameters and track all its results:
 * input_units
 * epochs
 * batch_size

In [0]:
!ls

conf  derby.log  eventlogs  ganglia  logs  metastore_db  preload_class.lst


In [0]:
for (input_units, epochs, batch_size) in [(64, 100, 64), (128, 200, 128), (256, 300, 128)]:
  keras_obj = KerasModel(X_train, input_units = input_units, loss="mse", optimizer="adam", metrics=["mse"])
  run_id = keras_obj.mlflow_run(X_train, y_train,  epochs=epochs, batch_size=batch_size, validation_split=.2)
  print(f"Finished running Keras training with run_id={run_id}")

Epoch 1/100
19/19 [==============================] - 1s 12ms/step - loss: 10447924.0000 - mse: 10447924.0000 - val_loss: 8180221.5000 - val_mse: 8180221.5000
Epoch 2/100
19/19 [==============================] - 0s 3ms/step - loss: 10072825.0000 - mse: 10072825.0000 - val_loss: 7840472.5000 - val_mse: 7840472.5000
Epoch 3/100
19/19 [==============================] - 0s 3ms/step - loss: 9702843.0000 - mse: 9702843.0000 - val_loss: 7480814.5000 - val_mse: 7480814.5000
Epoch 4/100
19/19 [==============================] - 0s 2ms/step - loss: 9278494.0000 - mse: 9278494.0000 - val_loss: 7054417.5000 - val_mse: 7054417.5000
Epoch 5/100
19/19 [==============================] - 0s 3ms/step - loss: 8782054.0000 - mse: 8782054.0000 - val_loss: 6580317.0000 - val_mse: 6580317.0000
Epoch 6/100
19/19 [==============================] - 0s 3ms/step - loss: 8235770.0000 - mse: 8235770.0000 - val_loss: 6102583.0000 - val_mse: 6102583.0000
Epoch 7/100
19/19 [==============================] - 0s 2ms/step 

Declared Utils class with utility methods: 
 Declared load_data(path, index_col=0) returns Pandas DataFrame for diagnostics 
 Declared get_training_data(df) returns X, y Pandas dataframe 
 Declared get_validation_data(df) returns val_x, val_y Pandas dataframe 
 Declared get_weather_and_forecast(df) returns Pandas Dataframe with dropped "power" columns 
 Declared wait_until_ready(model_name, model_version)

Declared PlotUtils class with utility methods: 
 Declared plot(model_uri, power_predictions, past_power_output) Plots a graph 
 Declared forecast_power(model_uri, df) Plots a graph

Using mlflow version 1.21.0


Declared KerasModel class with public methods: 
 Declared mlflow_run(model, X_train, y_train, **kwargs) returns MLflow run_id

# Register the best Keras model with the MLflow Model Registry

Now that a forecasting model has been trained and tracked with MLflow, the next step is to register it with the MLflow Model Registry. You can register and manage models using the MLflow UI.

## Part 1: Use a Neural Network model using Kera

### Create a new Registered Model

1. First, navigate to the MLflow Runs Sidebar by clicking the `Runs` icon in the Databricks Notebook UI.


2. Next, locate the MLflow Run corresponding to the Keras model training session, and open it in the MLflow Run UI by clicking the `View Run Detail` icon.

3. In the MLflow UI, scroll down to the `Artifacts` section and click on the directory named `model`. Click on the `Register Model` button that appears.

4. Then, select `Create New Model` from the drop-down menu, and input the following model name: `PowerForecastingModel`. Finally, click `Register`. This registers a new model called `PowerForecastingModel` and creates a new model version: `Version 1`.

After a few moments, the MLflow UI displays a link to the new registered model. Follow this link to open the new model version in the MLflow Model Registry UI.

### Explore the Model Registry UI Workflow

1. The Model Version page in the MLflow Model Registry UI provides information about `Version 1` of the registered forecasting model, including its author, creation time, and its current stage.

2. The Model Version page also provides a `Source Run` link, which opens the MLflow Run that was used to create the model in the MLflow Run UI. From the MLflow Run UI, you can access the `Source Notebook Link` to view a snapshot of the Databricks Notebook that was used to train the model.

To navigate back to the MLflow Model Registry, click the `Models` icon in the Databricks Workspace Sidebar. The resulting MLflow Model Registry home page displays a list of all the registered models in your Databricks Workspace, including their versions and stages.

Select the `PowerForecastingModel` link to open the Registered Model page, which displays all of the versions of the forecasting model.

### Add model descriptions

You can add descriptions to Registered Models as well as Model Versions: 
* Model Version descriptions are useful for detailing the unique attributes of a particular Model Version (e.g., the methodology and algorithm used to develop the model). 
* Registered Model descriptions are useful for recording information that applies to multiple model versions (e.g., a general overview of the modeling problem and dataset).

Add a high-level description to the registered power forecasting model by clicking the `Edit Description` icon, entering the following description, and clicking `Save`:

```
This model forecasts the power output of a wind farm based on weather data. The weather data consists of three features: wind speed, wind direction, and air temperature.
```

Next, click the `Version 1` link from the Registered Model page to navigate back to the Model Version page. Then, add a model version description with information about the model architecture and machine learning framework; click the `Edit Description` icon, enter the following description, and click `Save`:
```
This model version was built using Keras. It is a feed-forward neural network with one hidden layer.
```

### Perform a model stage transition

The MLflow Model Registry defines several model stages: `None`, `Staging`, `Production`, and `Archived`. Each stage has a unique meaning. For example, `Staging` is meant for model testing, while `Production` is for models that have completed the testing or review processes and have been deployed to applications. 

Users with appropriate permissions can transition models between stages. In private preview, any user can transition a model to any stage. In the near future, administrators in your organization will be able to control these permissions on a per-user and per-model basis.

If you have permission to transition a model to a particular stage, you can make the transition directly. If you do not have permission, you can request a stage transition from another user.

Click the `Stage` button to display the list of available model stages and your available stage transition options. Select `Transition to -> Production` and press `OK` in the stage transition confirmation window to transition the model to `Production`.

#### Now that the model has been registered and transitioned to `Production`, navigate to the "Integrate the model with the forecasting application" section of the quickstart.

In [0]:
model_name = "PowerForecastingModel"

# Integrate the model with the forecasting application

Now that you have trained and registered a power forecasting model with the MLflow Model Registry, the next step is to integrate it with an application. This application fetches a weather forecast for the wind farm over the next five days and uses the model to produce power forecasts. For example purposes, the application consists of a simple `forecast_power()` function (defined below) that is executed within this notebook. In practice, you may want to execute this function as a recurring batch inference job using the Databricks Jobs service.

The following **"Load versions of the registed model"** section demonstrates how to load model versions from the MLflow Model Registry for use in applications. Then, the **"Forecast power output with the production model"** section uses the `Production` model to forecast power output for the next five days.

## Load versions of the registered model

The MLflow Models component defines functions for loading models from several machine learning frameworks. For example, `mlflow.keras.load_model()` is used to load Keras models that were saved in MLflow format, and `mlflow.sklearn.load_model()` is used to load scikit-learn models that were saved in MLflow format.

These functions can load models from the MLflow Model Registry.

You can load a model by specifying its name (e.g., `PowerForecastingModel`) and version number (e.g., `1`). The following cell uses the `mlflow.pyfunc.load_model()` API to load `Version 1` of the registered power forecasting model as a generic Python function.

In [0]:
import mlflow.pyfunc

model_version_uri = "models:/{model_name}/1".format(model_name=model_name)

print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_version_uri))
model_version_1 = mlflow.pyfunc.load_model(model_version_uri)


Loading registered model version from URI: 'models:/PowerForecastingModel/1'


---------------------------------------------------------------------------
MissingSchema                             Traceback (most recent call last)
/databricks/python/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in cloud_storage_http_request(method, url, max_retries, backoff_factor, retry_codes, timeout, **kwargs)
    251     try:
--> 252         with _get_http_response_with_retries(
    253             method, url, max_retries, backoff_factor, retry_codes, timeout=timeout, **kwargs

/databricks/python/lib/python3.8/site-packages/mlflow/utils/rest_utils.py in _get_http_response_with_retries(method, url, max_retries, backoff_factor, retry_codes, **kwargs)
     73         http.mount("http://", adapter)
---> 74         response = http.request(method, url, **kwargs)
     75         return response

/databricks/python/lib/python3.8/site-packages/requests/sessions.py in request(self, method, url, params, data, headers, cookies, files, auth, timeout, allow_redirects, proxies, ho

You can also load a specific model stage. The following cell loads the `Production` stage of the power forecasting model.

In [0]:
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_production_uri))
model_production = mlflow.pyfunc.load_model(model_production_uri)

## Forecast power output with the production model

In this section, the production model is used to evaluate weather forecast data for the wind farm. The `PlotUtils.forecast_power()` class loads a version of the forecasting model from the specified URI and uses it to forecast power production over the next five days.

In [0]:
PlotUtils.forecast_power(model_production_uri, wind_farm_data)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2387317862618037> in <module>
----> 1 PlotUtils.forecast_power(model_production_uri, wind_farm_data)

NameError: name 'model_production_uri' is not defined

## Part 2: Use a Scikit-learn Randorm Forest Regressor to create the second model. 

This model, in real development environment, could be developed by another developer and registered in the model registry. 

Use MLflow to compare and then elect the best model between Keras and Random Forest for production.

# Create and deploy a new model version

The MLflow Model Registry enables you to create multiple model versions corresponding to a single registered model. By performing stage transitions, you can seamlessly integrate new model versions into your staging or production environments. Model versions can be trained in different machine learning frameworks (e.g., `scikit-learn` and `Keras`); MLflow's `python_function` provides a consistent inference API across machine learning frameworks, ensuring that the same application code continues to work when a new model version is introduced.

The following sections create a new version of the power forecasting model using scikit-learn, perform model testing in `Staging`, and update the production application by transitioning the new model version to `Production`.

## Create a new model version

Classical machine learning techniques are also effective for power forecasting. The following cell trains a random forest model using scikit-learn and registers it with the MLflow Model Registry via the `mlflow.sklearn.log_model()` function.

As above, we will try few different parameters and choose the best model.

In [0]:
params_list = [
        {"n_estimators": 100},
        {"n_estimators": 200},
        {"n_estimators": 300}]
#iterate over few different tuning parameters
for params in params_list:
  rfr = RFRModel.new_instance(params)
  print("Using paramerts={}".format(params))
  runID = rfr.mlflow_run(X_train, y_train, val_x, val_y, model_name)
  print("MLflow run_id={} completed with MSE={} and RMSE={}".format(runID, rfr.mse,rfr.rsme))

Using paramerts={'n_estimators': 100}
Validation MSE: 45241
Validation RMSE: 212


---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
<command-2387317862618041> in <module>
      7   rfr = RFRModel.new_instance(params)
      8   print("Using paramerts={}".format(params))
----> 9   runID = rfr.mlflow_run(X_train, y_train, val_x, val_y, model_name)
     10   print("MLflow run_id={} completed with MSE={} and RMSE={}".format(runID, rfr.mse,rfr.rsme))

<command-2387317862618003> in mlflow_run(self, X_train, y_train, val_x, val_y, model_name, run_name)
     42       # function to register the model with the MLflow Model Registry. This automatically
     43       # creates a new model version
---> 44       mlflow.sklearn.log_model(
     45         sk_model= self.model(),
     46         artifact_path="sklearn-model",

/databricks/python/lib/python3.8/site-packages/mlflow/sklearn/__init__.py in log_model(sk_model, artifact_path, conda_env, serialization_format, registered_mod

### Explore the Model Registry API Workflow

When a new model version is created with the MLflow Model Registry's Python APIs, the name and version information is printed for future reference. You can also navigate to the MLflow Model Registry UI to view the new model version. 

Define the new model version as a variable in the cell below.

In [0]:
model_version = 2 # If necessary, replace this with the version corresponding to the new scikit-learn model

Wait for the new model version to become ready.

In [0]:
Utils.wait_until_ready(model_name, model_version)

## Add a description to the new model version

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
client.update_model_version(
  name=model_name,
  version=model_version,
  description="This is the best model version is a random forest that was trained in scikit-learn."
)

## Test the new model version in `Staging`

Before deploying a model to a production application, it is often best practice to test it in a staging environment. The following cells transition the new model version to `Staging` and evaluate its performance.

In [0]:
client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage="Staging",
)

Evaluate the new model's forecasting performance in `Staging`

In [0]:
model_staging_uri = "models:/{model_name}/staging".format(model_name=model_name)
PlotUtils.forecast_power(model_staging_uri,wind_farm_data)

## Deploy the new model version to `Production`

After verifying that the new model version performs well in staging, the following cells transition the model to `Production` and use the exact same application code from the **"Forecast power output with the production model"** section to produce a power forecast. 

**The MLflow Model Model Registry automatically uses the latest production version of the specified model, allowing you to update your production models without changing any application code**.

In [0]:
client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage="Production"
)

In [0]:
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)
PlotUtils.forecast_power(model_production_uri,wind_farm_data)

# Delete models

When a model version is no longer being used, you can archive it or delete it. You can also delete an entire registered model; this removes all of its associated model versions.

### Workflow 1: Delete `Version 1` in the MLflow UI

To delete `Version 1` of the power forecasting model, open its corresponding Model Version page in the MLflow Model Registry UI. Then, select the drop-down arrow next to the version identifier and click `Delete`.

### Workflow 2: Delete `Version 1` using the MLflow API

The following cell permanently deletes `Version 1` of the power forecasting model. If you want to delete this model version, uncomment and execute the cell.

In [0]:
client.delete_model_version(name=model_name,version=1)

## Delete the power forecasting model

If you want to delete an entire registered model, including all of its model versions, you can use the `MlflowClient.delete_registered_model()` to do so. This action cannot be undone.

**WARNING: The following cell permanently deletes the power forecasting model, including all of its versions.** If you want to delete the model, uncomment and execute the cell.

In [0]:
#### REMOVE THIS CELL BEFORE DISTRIBUTING ####
client.delete_registered_model(name=model_name)